<a href="https://colab.research.google.com/github/WB-Jang/AI-RPA/blob/main/Committee-agent/Word_%EB%8F%99%EC%8B%9C%ED%8E%B8%EC%A7%91_Pdf_%ED%86%B5%ED%95%A9_%ED%94%8C%EB%9E%AB%ED%8F%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Committee 업무 중 The most time-consuming task인 문서 취합 효율화 플랫폼
- 기능 1 : 여러 Word 파일들을 한 화면에서 동시 편집 가능
- 기능 2 : 일부 수정된 Word 파일을 Local PC에 저장 혹은 작성 부서에 e-mail 전송
- 기능 3 : 최종 수정된 Word 파일들을 pdf로 통합
- 기능 4 : LLM을 활용하여, 오타 및 논리적 오류가 보이는 부분 Early-alert

In [3]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.4 MB/s eta 0:00:00


In [7]:
%%writefile app.py
# Streamlit 어플 구성
import streamlit as st
import os

def main():
  st.set_page_config(page_title="Word 취합 및 교정, Pdf 통합 플랫폼", layout='wide')

  if "analysis_cache" not in st.session_state:
    st.session_state.analysis_cache = {}
  if "summary_cache" not in st.session_state:
    st.session_state.summary_cache = {}

  st.sidebar.header("설정")
  folder = st.sidebar.text_input("Word 파일 폴더 경로", value="")
  output_pdf = st.sidebar.text_input("최종 통합 pdf 저장 경로", value=os.path.join(os.getcwd(), 'Integrated.pdf'))
  st.title("Word 문서 취합/교정/통합 PDF 생성 tool")

  if not folder or not os.path.isdir(folder):
    st.warning("좌측에 Word 파일들이 모여 있는 폴더 경로를 입력하세요.")
    return

  files = list_word_files(folder)
  if not files:
    st.warning(f"{folder} 폴더에서 .doc / .docx 파일을 찾지 못했습니다.")
    return

  st.sidebar.markdown("---")

if __name__ == "__main__":
  main()

Writing app.py


In [9]:
!pip install pyngrok

In [10]:
import threading
import subprocess
import time
from pyngrok import ngrok

def run_stramlit():
  cmd = ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
  subprocess.run(cmd)

thread = threading.Thread(target=run_stramlit, daemon=True)
thread.start()

time.sleep(5)

NGROK_AUTH_TOKEN = "본인 토큰"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(8501, "http")
print("외부 접속 URL : ", public_url)

ERROR:pyngrok.process.ngrok:t=2025-11-18T07:33:15+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.